# This Notebook tries to Predict number of Stickers Sold

Let's start by loading the data for exploration.

In [3]:
# Necessary libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
path = "../../../Data/playground-series-s5e1/"
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")

A qu